In [1]:
import os
import numpy as np
from PIL import Image
from typing import Tuple, List
from abc import ABC, abstractmethod
from scipy.optimize import differential_evolution

class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class DistanceFunction:
    def __call__(self, image: np.ndarray, template: np.ndarray) -> float:
        return np.mean((image - template) ** 2)

class TemplateFactory:
    def __init__(self, r_min: int, r_max: int, size: Tuple[int, int]):
        self.r_min = r_min
        self.r_max = r_max
        self.size = size

    def generate_template(self, positions: List[Tuple[int, int, int]]) -> np.ndarray:
        template = np.zeros(self.size)
        for x, y, r in positions:
            for i in range(self.size[0]):
                for j in range(self.size[1]):
                    if (i - x) ** 2 + (j - y) ** 2 <= r ** 2:
                        template[i, j] = 1
        return template

    def generate_templates(self, max_objects: int) -> List[np.ndarray]:
        templates = []
        for count in range(1, max_objects + 1):
            pos = [(np.random.randint(0, self.size[0]),
                    np.random.randint(0, self.size[1]),
                    np.random.randint(self.r_min, self.r_max + 1)) for _ in range(count)]
            templates.append(self.generate_template(pos))
        return templates

class OptimizationStrategy(ABC):
    @abstractmethod
    def optimize(self, image: np.ndarray, templates: List[np.ndarray], distance_fn: DistanceFunction) -> int:
        pass

class MonteCarloGradientDescent(OptimizationStrategy):
    def __init__(self, iterations: int = 50):
        self.iterations = iterations

    def optimize(self, image: np.ndarray, templates: List[np.ndarray], distance_fn: DistanceFunction) -> int:
        min_distance = float('inf')
        best_index = 0
        for i, template in enumerate(templates):
            dist = distance_fn(image, template)
            if dist < min_distance:
                min_distance = dist
                best_index = i
        return best_index + 1

class PeanoMGGOptimization(OptimizationStrategy):
    def __init__(self, r_min: int, r_max: int, size: Tuple[int, int], max_objects: int = 10):
        self.r_min = r_min
        self.r_max = r_max
        self.size = size
        self.max_objects = max_objects
        self.template_factory = TemplateFactory(r_min, r_max, size)

    def optimize(self, image: np.ndarray, templates: List[np.ndarray], distance_fn: DistanceFunction) -> int:
        def loss(flat_params):
            count = len(flat_params) // 3
            positions = [(int(flat_params[3*i]), int(flat_params[3*i+1]), int(flat_params[3*i+2]))
                         for i in range(count)]
            template = self.template_factory.generate_template(positions)
            return distance_fn(image, template)

        bounds = []
        for _ in range(self.max_objects):
            bounds.extend([
                (0, self.size[0] - 1),
                (0, self.size[1] - 1),
                (self.r_min, self.r_max)
            ])

        result = differential_evolution(loss, bounds, maxiter=30, polish=True, disp=False)
        best_positions = [(int(result.x[3*i]), int(result.x[3*i+1]), int(result.x[3*i+2]))
                          for i in range(self.max_objects)]
        best_template = self.template_factory.generate_template(best_positions)

        distances = [distance_fn(image, t) for t in templates]
        return int(np.argmin(distances)) + 1

class CalculationManager(metaclass=SingletonMeta):
    def __init__(self, log_file='log.txt'):
        self.log_file = log_file
        self.results = []

    def log(self, image_name: str, result: int, distance: float, method_name: str):
        self.results.append((image_name, result, distance, method_name))
        with open(self.log_file, 'a') as f:
            f.write(f'{image_name}, {result}, {distance:.6f}, {method_name}\n')

class CellCounter:
    def __init__(self, strategy: OptimizationStrategy, template_factory: TemplateFactory):
        self.strategy = strategy
        self.template_factory = template_factory
        self.distance_fn = DistanceFunction()
        self.manager = CalculationManager()

    def __call__(self, folder_path: str):
        templates = self.template_factory.generate_templates(max_objects=10)
        method_name = self.strategy.__class__.__name__
        for filename in os.listdir(folder_path):
            if filename.endswith('.png') or filename.endswith('.jpg'):
                path = os.path.join(folder_path, filename)
                image = Image.open(path).convert('L')
                image_resized = image.resize(self.template_factory.size)
                image_array = np.array(image_resized) / 255.0
                predicted = self.strategy.optimize(image_array, templates, self.distance_fn)
                dist = self.distance_fn(image_array, templates[predicted - 1])
                self.manager.log(filename, predicted, dist, method_name)



In [3]:
template_factory = TemplateFactory(r_min=1, r_max=3, size=(64, 64))

strategy = MonteCarloGradientDescent(iterations=50)
counter = CellCounter(strategy, template_factory)
counter('D:\mfti\python\second\HT4\data')


strategy = PeanoMGGOptimization(r_min=1, r_max=3, size=(64, 64), max_objects=3)
counter = CellCounter(strategy, template_factory)
counter('D:\mfti\python\second\HT4\data')